---
title: IDs from ARTHEMIS
order: 13
---

See following notebooks for details:

- [Magnetic field data](./mag.ipynb)
- [State data](./state.ipynb)

## Background

ARTEMIS spacecrafts will be exposed in the solar wind at 1 AU during its orbits around the Moon. So it's very interesting to look into its data.

- For time inteval for THEMIS-B in solar wind, see [Link](https://omniweb.gsfc.nasa.gov/ftpbrowser/themis_b_sw.txt)
- For time inteval for THEMIS-C in solar wind, see [Link](https://omniweb.gsfc.nasa.gov/ftpbrowser/themis_c_sw.txt)


## Setup

Need to run command in shell first as `pipeline` is project-specific command

```{sh}
kedro pipeline create themis
```

To get candidates data, run `kedro run --from-inputs=jno.feature_1s --to-outputs=candidates.jno_1s`

In [1]:
# | code-summary: import all the packages needed for the project
# | output: hide
# | export
import polars as pl
from kedro.pipeline import Pipeline, node
from kedro.pipeline.modular_pipeline import pipeline

from ids_finder import PARAMS
from ids_finder.pipelines.themis.mag import create_pipeline as create_mag_data_pipeline
from ids_finder.pipelines.themis.state import create_pipeline as create_state_data_pipeline
from ids_finder.pipelines.default.mission import create_combined_data_pipeline

from typing import Optional

[11/21/23 16:37:49] WARNING  TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook    ]8;id=299535;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=433809;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py#109\109]8;;\
                             mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter                  
                             console)                                                                              
                                                                                                                   

In [ ]:
#| default_exp pipelines/themis/pipeline

## Solar wind pipeline

In [ ]:
#| export
from ids_finder.utils.basic import filter_tranges_df

def filter_sw_events(events: pl.LazyFrame, sw_state: pl.LazyFrame) -> pl.LazyFrame:
    
    start, end = sw_state.select(['start', 'end']).collect()
    sw_events = filter_tranges_df(events.collect(), (start, end))
    
    return sw_events

def create_sw_events_pipeline(
    sat_id,
    tau: int = 60,
    ts_mag: int = 1,
    
):
  
    ts_mag_str = f"ts_{ts_mag}s"
    tau_str = f"tau_{tau}s"
    
    node_filter_sw_events = node(
        filter_sw_events,
        inputs=[
            f"events.{sat_id}_{ts_mag_str}_{tau_str}",
            f"{sat_id}.STATE.inter_data_sw",
        ],
        outputs=f"events.{sat_id}_sw_{ts_mag_str}_{tau_str}"
        
    )

    nodes = [node_filter_sw_events]
    return pipeline(nodes)

## Pipelines

In [ ]:
# | export
def create_pipeline(
    sat_id="THB",
    params: Optional[dict] = None,
):
    
    if params is None:
        params = PARAMS
    tau = params["tau"]
    ts_state = params[sat_id]["STATE"]["time_resolution"]
    ts_mag = params[sat_id]["MAG"]["time_resolution"]
    ts_state_str = f"ts_{ts_state}s"


    input_combined_data = {
        f"{sat_id}.STATE.primary_data_{ts_state_str}": f"OMNI.LowRes.primary_data_{ts_state_str}"
    }
    
    node_combined_data = pipeline(
        create_combined_data_pipeline(sat_id),
        inputs=input_combined_data,
    )

    return (
        create_mag_data_pipeline(sat_id)
        + create_state_data_pipeline(sat_id)
        + node_combined_data
        + create_sw_events_pipeline(sat_id, tau=tau, ts_mag=ts_mag)
    )